In [1]:
# Optimization 3--ran out of dimensions that could be reduced. Added hidden layers to attempt to bring up the accuracy

In [2]:

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras

In [3]:
#  Import and read the charity_data.csv.
# 3 hours waiting for tensor flow to install on jupyter, went with colaboratory
url = 'https://raw.githubusercontent.com/Jermov/deep-learning-challenge/main/Resources/charity_data.csv'
df = pd.read_csv(url)

In [4]:
# Check application_df for null/na values
df.isnull().sum()

EIN                       0
NAME                      0
APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
STATUS                    0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64

In [5]:

# Check application_df for duplicated values
df.duplicated().sum()

0

In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df.drop(columns = ['EIN', 'NAME','STATUS'], inplace=True)
df


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,N,5000,1


In [7]:
# Determine the number of unique values in each column
for i in df.columns:
    print(i, len(df[i].unique()))

APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [8]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = df['APPLICATION_TYPE'].value_counts()
app_counts


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application 500 seemed to not do much for the model changed binning to <800 for opt2
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts<800].index)
# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

In [10]:
# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [11]:
 # You may find it helpful to look at CLASSIFICATION value counts >1
ask_amt_count =df['ASK_AMT'].value_counts().loc[lambda x : x >1]
ask_amt_count

5000      25398
10478         3
15583         3
63981         3
6725          3
          ...  
100000        2
8181          2
24746         2
45793         2
240780        2
Name: ASK_AMT, Length: 152, dtype: int64

In [12]:
df['ranges'] = pd.cut(df['ASK_AMT'], [5000, 25000,100000,500000,5000000000])
df['ranges'].value_counts()

(25000, 100000]         2686
(100000, 500000]        2304
(500000, 5000000000]    2224
(5000, 25000]           1684
Name: ranges, dtype: int64

In [13]:
# Look at CLASSIFICATION value counts for binning
classification_counts = df['CLASSIFICATION'].value_counts()

In [14]:

# You may find it helpful to look at CLASSIFICATION value counts >1
df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [15]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = list(classification_counts[classification_counts<1000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [16]:
df.drop(columns = ['ASK_AMT'], inplace=True)

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummy = pd.get_dummies(df)
app_dummy.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,"ranges_(5000, 25000]","ranges_(25000, 100000]","ranges_(100000, 500000]","ranges_(500000, 5000000000]"
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
2,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0


In [18]:

# Split our preprocessed data into our features and target arrays
X = app_dummy.drop('IS_SUCCESSFUL', axis=1).values
y = app_dummy['IS_SUCCESSFUL'].values


In [19]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)


In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Compile, Train and Evaluate the Model

In [22]:

# Get the input feature/ shape
X_train_scaled.shape


(25724, 42)

In [23]:
input_features = X_train_scaled.shape[1]

In [24]:
# Automated Neural Network Optimization


In [25]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.2 MB/s 
     |████████████████████████████████| 1.6 MB 36.3 MB/s 


In [26]:

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    opt3 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    opt3.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value= 90,
        step=5), activation=activation, input_dim=input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 40)):
        opt3.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))
    
    opt3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    opt3.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return opt3


In [27]:

# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    
    max_epochs=50,
    hyperband_iterations=2)


In [28]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 04m 24s]
val_accuracy: 0.5343440175056458

Best val_accuracy So Far: 0.7338775396347046
Total elapsed time: 02h 27m 35s


In [29]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters()[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 76,
 'num_layers': 22,
 'units_0': 16,
 'units_1': 11,
 'units_2': 16,
 'units_3': 11,
 'units_4': 21,
 'units_5': 21,
 'units_6': 1,
 'units_7': 21,
 'units_8': 21,
 'units_9': 11,
 'units_10': 21,
 'units_11': 26,
 'units_12': 11,
 'units_13': 21,
 'units_14': 6,
 'units_15': 11,
 'units_16': 6,
 'units_17': 6,
 'units_18': 26,
 'units_19': 21,
 'units_20': 16,
 'units_21': 21,
 'units_22': 11,
 'units_23': 11,
 'units_24': 6,
 'units_25': 16,
 'units_26': 26,
 'units_27': 6,
 'units_28': 26,
 'units_29': 21,
 'units_30': 11,
 'units_31': 16,
 'units_32': 6,
 'units_33': 11,
 'units_34': 26,
 'units_35': 11,
 'units_36': 11,
 'units_37': 1,
 'units_38': 21,
 'units_39': 6,
 'tuner/epochs': 6,
 'tuner/initial_epoch': 2,
 'tuner/bracket': 3,
 'tuner/round': 1,
 'tuner/trial_id': '0023'}

In [30]:

# Evaluate best model against full test data
best_model = tuner.get_best_models()[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 1s - loss: 0.5638 - accuracy: 0.7339 - 1s/epoch - 4ms/step
Loss: 0.5638139843940735, Accuracy: 0.7338775396347046


In [31]:
# Export our model to HDF5 file

best_model.save('../h5_fileswq /AlphabetSoupCharity_Optimization3.h5')


 
 
